# Encoding with Altair

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_3/encoding-with-altair.ipynb)

Altairを使って、データから可視化への変換をもう少し練習してみましょう。

In [ ]:
# ライブラリのインストール

!pip install altair
!pip install vega_datasets

In [3]:
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

## Gapminderのデータの読み込み

1955年から2005年までの期間における、いくつかの国の健康と人口データを可視化してみましょう。このデータはGapminder Foundationによって収集され、ハンス・ロスリングのTEDトークで共有されたものです。

まず、vega-datasetsコレクションからデータセットをPandasのデータフレームにロードします。

In [7]:
data = vega_data.gapminder()

データを確認します。

In [8]:
data.shape

(693, 6)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         693 non-null    int64  
 1   country      693 non-null    object 
 2   cluster      693 non-null    int64  
 3   pop          693 non-null    int64  
 4   life_expect  693 non-null    float64
 5   fertility    693 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 32.6+ KB


In [10]:
data.head()

,year,country,cluster,pop,life_expect,fertility
0,1955,Afghanistan,0,8891209,30.332,7.7
1,1960,Afghanistan,0,9829450,31.997,7.7
2,1965,Afghanistan,0,10997885,34.020,7.7
3,1970,Afghanistan,0,12430623,36.088,7.7
4,1975,Afghanistan,0,14132019,38.438,7.7


各国（`country`）・各年（`year`, 5年間隔）ごとに、女性一人当たりの子供の数（`fertility`）、平均寿命（`life_expect`）、総人口（`pop`）を測定しています。これがデータ変数です。

また、クラスタのフィールドには、整数が表示されています。これは何を表しているのでしょうか？データを可視化しながら、これを明らかにしていきましょう。

さらに、2000年の値だけに絞り込んで、より小さなデータフレームを作成します。

In [11]:
data2000 = data[data["year"] == 2000]

In [13]:
data2000.head()

,year,country,cluster,pop,life_expect,fertility
9,2000,Afghanistan,0,23898198,42.129,7.4792
20,2000,Argentina,3,37497728,74.340,2.3500
31,2000,Aruba,3,69539,73.451,2.1240
42,2000,Australia,4,19164620,80.370,1.7560
53,2000,Austria,1,8113413,78.980,1.3820


## データ変数の性質

Altairではテキストを直接扱うことはできません。また、間隔尺度と比例尺度を量的尺度としてまとめて扱っています。

そのため、ここではデータ変数の性質のうち、名義尺度、順序尺度、量的尺度（間隔尺度、比例尺度）について見ていきます。

### 名義尺度（Nominal, N）

名義尺度のデータ変数は、はカテゴリー名で構成されます。

値Aと値Bは同じか違うか（A = B）というような、値の等質性を比較することができます。上のデータセットでは、`country`は名義尺度です。

位置、色相（青、赤、緑など）、形状を見れば、値が同じか異なるかがすぐに分かります。しかし、大きさを使用して名義尺度を変換すると、存在しない値間の順位や大きさの違いを示唆し、誤解を招く恐れがあります。

### 順位尺度（Ordinal, O）

順序データは、特定の順序を持つ値で構成されます。

値Aは値Bの前か後か?（A < B）というような比較をすることができます。上のデータセットでは、5年間隔の`year`を順序尺度として扱うことができます。

順序尺度のデータ変数を可視化する場合は、順序の感覚に合わせる必要があります。位置、大きさ、明るさなどが適切で、色相（知覚的に順序がないか直感的には分からない）はあまり適切ではありません。

### 量的尺度（Quantitative, Q）

量的尺度では、数値の違いを測定することができます。量的尺度には、2種類あります。

間隔尺度では、点間の距離（区間）を測定することができます。値Bから値Aまでの距離は何メートルk？（A - B）のような計算をすることができます。

比例尺度では、ゼロに意味があるので、比率を計算することができます。値Aは値Bの何％か？（A / B）のような計算が可能です。

上のデータセットでは、`year`は間隔尺度（yearの値「0」は一つの目安であり、量が存在しないことを意味しない）、`fertility`と`life_expect`は比例尺度です。

量的尺度は、位置、サイズ、明るさなどを使用して可視化することができます。ベースラインがゼロの軸は、比例尺度の比較には必須ですが、間隔尺度の比較では省略することが可能です。

### 補足

これらのデータ型は相互に排他的ではなく、階層を形成しています。量的尺度は順序尺度を含み、順序尺度は名義尺度を含みます。

そのため、変数は1つの性質のみを持っているわけではありません。変数が数値で表現されていても、名義尺度や順序尺度として扱うことができます。例えば、年齢（10歳、20歳など）は、名義尺度（未成年、成人）、順序尺度（年ごとにグループ化）、定量尺度として扱うことができます。

次に、これらのデータ変数をどのように可視化するかを見てみましょう。

## 視覚変数

Altairでは、視覚記号をmarkといいます。また、視覚変数はencoding channelに含まれます。

主な視覚変数は次の通りです。

* `x`: x軸の位置
* `y`: y軸の位置
* `size`: 視覚記号（mark）の大きさ。視覚記号によって、面積または長さに対応する
* `color`: 色
* `opacity`: 不透明度。0（透明）–1（不透明）の値をとる
* `shape`: 視覚記号の形

その他、視覚変数ではなく、（次回勉強する）デザイン原則もencoding channelに含まれます。

* `tooltip`: 視覚記号の上にマウスを置いたときに表示される内容
* `order`: 視覚記号の描画順序
* `column`: 複数の可視化を左右に描画
* `row`: 複数の可視化を上下に描画

### X

`x`は、視覚記号のx座標を設定します。軸とタイトルはデフォルトで表示されます。量的尺度を選択すると、軸のスケールが連続した直線になります。

In [16]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q')
)

/Users/shu/.virtualenvs/datatools/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

### Y

`y`は、視覚記号のy座標を設定します。ここでは、順序尺度のデータ（O）を使用しています。その結果、値ごとの離散軸が得られました。

In [19]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:O')
)

alt.Chart(...)

> OをQにしたらどうなるでしょう？

`cluster`のかわりに`life_expect`を量的尺度として追加すると、両軸が線形スケールの散布図になります。

In [21]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q')
)

alt.Chart(...)

デフォルトでは、線形定量尺度の軸は、比例尺度のデータを比較するための適切なベースラインを確保するためにゼロを含んでいます。

しかし、場合によっては、ゼロの基準線は無意味であったり、区間比較に焦点を当てたいことがあります。ゼロを自動的に含まないようにするには、`scale`を指定します。

In [25]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q', scale=alt.Scale(zero=False)),
    alt.Y('life_expect:Q', scale=alt.Scale(zero=False))
)

alt.Chart(...)

これで、軸にゼロが含まれなくなりました。軸の端は5や10の倍数のようなniceな数字に自動的に調整されるため、いくらかの余白が残っています

> 上記のscale属性に`nice=False`を追加するとどうなるでしょうか？試してみましょう。

In [27]:
# your code goes here

### Size

`size`は、視覚記号の大きさを設定します。この変数の意味は、視覚記号の種類によって異なります。点（`point`）の場合は、点の面積に対応します。

`size`に人口（`pop`）を指定することによって、散布図を拡張してみましょう。その結果、このグラフには点の大きさを解釈するための凡例（legend）も含まれるようになりました。

In [28]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q')
)

alt.Chart(...)

場合によっては、デフォルトのサイズ範囲ではうまくいかないことがあります。サイズの範囲を指定するには、scale属性の`range`パラメータに、最小と最大のサイズを示す配列を渡します。

In [29]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0, 1000]))
)

alt.Chart(...)

### Color and Opacity